In [26]:
modules = dict()
states = dict()

with open('input') as f:
    for line in f:
        line = line.strip().split()
        source = line[0]
        kind = "broadcaster"
        if source[0] == '%':
            kind = "flipflop"
            source = source[1:]
            states[source] = "off"
        if source[0] == '&':
            kind = "conjuction"
            source = source[1:]
            states[source] = dict()
        destinations = line[2:]
        new_dest = []
        for dst in destinations:
            if dst[-1] == ',':
                new_dest.append(dst[0:-1])
            else:
                new_dest.append(dst)
        modules[source] = [kind, new_dest]

for mod_name in modules.keys():
    mod = modules[mod_name]
    destinations = mod[1]
    for dst in destinations:
        if dst not in modules:
            continue
        if modules[dst][0] == "conjuction":
            current_state = states[dst]
            current_state[mod_name] = 'L'
            states[dst] = current_state

total_low = 0
total_high = 0

searching_for = dict()

i = 0

rz_val = None
lf_val = None
br_val = None
fk_val = None

found = False

while found is False:
    i += 1
    round_low = 1
    round_high = 0
    lows_to_propagate = [["button", "broadcaster"]]
    highs_to_propagate = []

    while len(lows_to_propagate) > 0 or len(highs_to_propagate) > 0:
        new_lows_to_propagate = []
        new_highs_to_propagate = []

        for [who, where] in lows_to_propagate:
            if modules[where][0] == "broadcaster":
                for next in modules[where][1]:
                    # print(where, "-low->", next)
                    if next in modules:
                        new_lows_to_propagate.append([where, next])
                    else:
                        round_low += 1


            if modules[where][0] == "flipflop":
                if states[where] == 'off':
                    states[where] = 'on'
                    for next in modules[where][1]:
                        # print(where, "-high->", next)
                        if next in modules:
                            new_highs_to_propagate.append([where, next])
                        else:
                            round_high += 1
                else:
                    states[where] = 'off'
                    for next in modules[where][1]:
                        # print(where, "-low->", next)
                        if next in modules:
                            new_lows_to_propagate.append([where, next])
                        else:
                            round_low += 1

            if modules[where][0] == "conjuction":
                state = states[where]
                state[who] = 'L'
                states[where] = state
                allHigh = True
                for key in state:
                    if state[key] == 'L':
                        allHigh = False
                        break
                if allHigh is True:
                    for next in modules[where][1]:
                        # print(where, "-low->", next)
                        if next in modules:
                            new_lows_to_propagate.append([where, next])
                        else:
                            round_low += 1
                else:
                    for next in modules[where][1]:
                        # print(where, "-high->", next)
                        if next in modules:
                            new_highs_to_propagate.append([where, next])
                        else:
                            round_high += 1

        for [who, where] in highs_to_propagate:
            if modules[where][0] == "conjuction":
                state = states[where]
                state[who] = 'H'
                states[where] = state
                allHigh = True
                for key in state.keys():
                    if state[key] == 'L':
                        allHigh = False
                        break
                if allHigh is True:
                    for next in modules[where][1]:
                        if next in modules:
                            new_lows_to_propagate.append([where, next])
                        else:
                            round_low += 1
                        # print(where, "-low->", next)
                else:
                    for next in modules[where][1]:
                        if next in modules:
                            new_highs_to_propagate.append([where, next])
                        else:
                            round_high += 1
                        # print(where, "-high->", next)

        lows_to_propagate = new_lows_to_propagate
        highs_to_propagate = new_highs_to_propagate
        for [who, where] in new_highs_to_propagate:
            if who == 'rz' and rz_val is None:
                rz_val = i
            if who == 'lf' and lf_val is None:
                lf_val = i
            if who == 'br' and br_val is None:
                br_val = i
            if who == 'fk' and fk_val is None:
                fk_val = i
            if rz_val is not None and lf_val is not None and br_val is not None and fk_val is not None:
                found = True
                print(rz_val * lf_val * br_val * rz_val)
                break
        if found is True:
            break
        round_low += len(lows_to_propagate)
        round_high += len(highs_to_propagate)
    total_low += round_low
    total_high += round_high


249570720246803
